In [1]:
import numpy as np
import numpy.linalg as la

In [2]:
#SVD algorithms
def calculate_eigh_ATA(A):
    '''
        Calculate the eigenvalues and eigenvectors of matrix A^T.A
        Arguments:
            A: numpy array - the image
        Returns:
            eigenvalues: numpy array
            eigenvectors: numpy array
    '''
    ATA = np.dot(A.T, A)
    eigenvalues, eigenvectors = la.eigh(ATA)
    eigenvalues = np.maximum(eigenvalues, 0.)

    #Sort descending
    sorted_index = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[sorted_index]
    eigenvectors = eigenvectors[:, sorted_index]

    return eigenvalues, eigenvectors

def calculate_svd(A):
    '''
        Using SVD to calculate U, sigma and V^T matrices of matrix A
        Arguments:
            A: numpy array - the image
        Returns:
            U: numpy array
            sigma: numpy array
            V_T: numpy array
    '''
    m = A.shape[0]
    n = A.shape[1]

    #Check to know calculate U or V^T first
    if m >= n:
        eigenvalues, eigenvectors = calculate_eigh_ATA(A.T)

        sigma = np.zeros([m, n])
        for i in range(min(m, n)):
            sigma[i][i] = max(eigenvalues[i], 0.)
        sigma = np.maximum(np.sqrt(sigma), 0)

        U = eigenvectors

        V = np.zeros([n, n])
        for i in range(n):
            V[:, i] = np.dot(A.T, U[:, i]) / sigma[i][i]
        V_T = V.T
    else:
        eigenvalues, eigenvectors = calculate_eigh_ATA(A)

        sigma = np.zeros([m, n])
        for i in range(min(m, n)):
            sigma[i][i] = max(eigenvalues[i], 0.)
        sigma = np.maximum(np.sqrt(sigma), 0)

        V = eigenvectors
        V_T = V.T

        U = np.zeros([m, m])
        for i in range(m):
            U[:, i] = np.dot(A, V[:, i]) / sigma[i][i]

    return U, sigma, V_T


def find_A_approx(A, rank):
    '''
        Calculate the matrix A_approximately of A with rank using SVD
        Arguments:
            A: numpy array - the image
            rank: int - the rank of the approximate matrix,
                the greater the rank is the more accuracy the approximate image is
        Returns:
            result: numpy array - the approximately image
            error: float - the error of the approximate image
    '''
    U, sigma, V_T = calculate_svd(A)
    #Approximate matrix with rank
    new_A = U[:, :rank] @ sigma[:rank, :rank] @ V_T[:rank, :]
    #Calculate error
    if rank < min(A.shape[0], A.shape[1]):
      error = np.sum(sigma[rank:, :])/ np.sum(sigma)
    else: 
      error = 0.
    return new_A, error

In [10]:
A = np.array([[1, 0, 1],
              [-2, 1, 0]])

In [12]:
while True:
  try:
    rank = int(input("Input rank: "))
    if rank > min(A.shape[0], A.shape[1]):
      print('The number must be lower! Try again!')
    elif rank < 0:
      print('The number must be positive! Try again!')
    else:
      break
  except ValueError:
    print('Please input a number!. Try again!')
  except:
    print('Something went wrong')
new_A, error = find_A_approx(A, rank)
print('Reconstruct A matrix with rank', rank, '\n', new_A)
print("Error:", error)

Reconstruct A matrix with rank 2 
 [[ 1.00000000e+00 -9.84273791e-17  1.00000000e+00]
 [-2.00000000e+00  1.00000000e+00 -9.84273791e-17]]
Error: 0.0
